In [ ]:
import json
import pandas as pd
import sqlite3

conn  = sqlite3.connect('merged_dataset_2.db')
query = "SELECT * FROM datasets_fixed"
df = pd.read_sql_query(query, conn)
conn.close()

In [ ]:
df = df.drop(columns=['level_0', 'index'])
df.head()

In [ ]:
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# temp = '''
# is_vpn
# is_proxy
# is_tor
# is_relay
# is_hosting
# '''
# temp = temp.replace('0', '')
# temp = temp.split('\n')

# temp2 = []
# for k in temp:
#   if k != '':
#     plt.figure()
#     sns.countplot(x = k.strip(), hue='source', data=df)

In [ ]:
df['is_hosting_filled'] = df['is_hosting'].fillna(0)
df['is_hosting_filled'] = df['is_hosting_filled'].astype(int)
df.drop(columns=['is_hosting'], inplace=True)
df.rename(columns={'is_hosting_filled': 'is_hosting'}, inplace=True)

df['formatted_output'] = df.apply(lambda row: f"method: {row['method'].lower()}\nheaders: {row['header_no_json']}\nis_vpn:{'true' if row['is_vpn'] == 1 else 'false'}\nis_proxy:{'true' if row['is_proxy'] == 1 else 'false'}\nis_tor:{'true' if row['is_tor'] == 1 else 'false'}\nis_relay:{'true' if row['is_relay'] == 1 else 'false'}\nis_hosting:{'true' if row['is_hosting'] == 1 else 'false'}\n", axis=1)

df.head()
df.info()
df['formatted_output'][0]

In [ ]:
import json
import pandas as pd
import sqlite3

conn  = sqlite3.connect('dataset_for_second_testing_2.db')
query = "SELECT * FROM dataset_for_second_testing_2"
df_validation = pd.read_sql_query(query, conn)
conn.close()

df_validation.head()

In [ ]:
df.value_counts(['source'])
# df_validation.value_counts(['source'])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

bots = df[df['source'] == 'bot']
humans = df[df['source'] == 'human']

bots_validation_split = bots.sample(n=200, random_state=42)
humans_validation_split = humans.sample(n=200, random_state=42)

df_validation_split = pd.concat([bots_validation_split, humans_validation_split])
y_validation_split = pd.concat([bots_validation_split, humans_validation_split])
y_validation_split = y_validation_split['source']

df = df.drop(y_validation_split.index)
y = df['source']

df_validation_new = df_validation
y_validation_new = df_validation['source']

In [ ]:
# df.head()
# df.value_counts(['source'])
# y

# df_validation_split.head()
# df_validation_split.value_counts(['source'])
# y_validation_split

# df_validation_new.head()
# df_validation_new.value_counts(['source'])
# y_validation_new

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
import zipfile
import glob

plt.rcParams.update({
    'font.size': 24,           # Base font size
    'axes.titlesize': 14,      # Title font size
    'axes.labelsize': 14,      # Axis label font size
    'xtick.labelsize': 12,     # X tick labels
    'ytick.labelsize': 12      # Y tick labels
})


models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "Linear SVC": LinearSVC(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Multinomial Naive Bayes": MultinomialNB(),
    "Gaussian Naive Bayes": GaussianNB(),
    "MLP Classifier": MLPClassifier(random_state=42, max_iter=1000),
}

def evaluate_model_graph(
    X_text_vec,
    y,
    X_text_vec2,
    y_test_test1,
    X_text_vec3,
    y_test_test2,
    models,
    vectorizer_name
):
    print(f"Vectorizer: {vectorizer_name}")
    X_train, X_test, y_train, y_test = train_test_split(X_text_vec, y, test_size=0.75, random_state=42)

    table_data = []

    for name, model in models.items():
        # Train
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        train_accuracy = model.score(X_train, y_train)
        train_f1_weighted = classification_report(y_train, y_pred_train, output_dict=True)['weighted avg']['f1-score']

        # Validation
        y_pred_val = model.predict(X_test)
        val_accuracy = model.score(X_test, y_test)
        val_f1_weighted = classification_report(y_test, y_pred_val, output_dict=True)['weighted avg']['f1-score']

        # Test1
        y_pred_test1 = model.predict(X_text_vec2)
        test1_accuracy = model.score(X_text_vec2, y_test_test1)
        test1_f1_weighted = classification_report(y_test_test1, y_pred_test1, output_dict=True)['weighted avg']['f1-score']

        # Test2
        y_pred_test2 = model.predict(X_text_vec3)
        test2_accuracy = model.score(X_text_vec3, y_test_test2)
        test2_f1_weighted = classification_report(y_test_test2, y_pred_test2, output_dict=True)['weighted avg']['f1-score']

        fig, axes = plt.subplots(2, 2, figsize=(10, 10))
        ConfusionMatrixDisplay.from_predictions(y_train, y_pred_train, ax=axes[0, 0], cmap='Blues')
        axes[0, 0].set_title(f'{name} - Train')
        ConfusionMatrixDisplay.from_predictions(y_test, y_pred_val, ax=axes[0, 1], cmap='Blues')
        axes[0, 1].set_title(f'{name} - Validation')
        ConfusionMatrixDisplay.from_predictions(y_test_test1, y_pred_test1, ax=axes[1, 0], cmap='Blues')
        axes[1, 0].set_title(f'{name} - Test 1')
        ConfusionMatrixDisplay.from_predictions(y_test_test2, y_pred_test2, ax=axes[1, 1], cmap='Blues')
        axes[1, 1].set_title(f'{name} - Test 2')
        plt.tight_layout()
        plt.savefig(f'confusion_{name.replace(" ", "_").lower()}.png')
        plt.close(fig)

        table_data.append([
            name,
            f"{train_accuracy * 100:.2f}%",
            f"{train_f1_weighted * 100:.2f}%",
            f"{val_accuracy * 100:.2f}%",
            f"{val_f1_weighted * 100:.2f}%",
            f"{test1_accuracy * 100:.2f}%",
            f"{test1_f1_weighted * 100:.2f}%",
            f"{test2_accuracy * 100:.2f}%",
            f"{test2_f1_weighted * 100:.2f}%"
        ])

    headers = [
        "Model",
        "Train Acc", "Train F1",
        "Val Acc", "Val F1",
        "Test1 Acc", "Test1 F1",
        "Test2 Acc", "Test2 F1"
    ]

    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    image_files = glob.glob("confusion_*.png")

    with zipfile.ZipFile("confusion_matrices.zip", "w") as zipf:
        for img in image_files:
            zipf.write(img)
    print("All confusion matrix images have been zipped into confusion_matrices.zip")


vectorizer_count = CountVectorizer(min_df=0.05, max_df=0.15, ngram_range=(2, 2), lowercase=False)
X_text_vec = vectorizer_count.fit_transform(df['formatted_output']).toarray()
X_text_vec2 = vectorizer_count.transform(df_validation_split['formatted_output']).toarray()
X_text_vec3 = vectorizer_count.transform(df_validation_new['formatted_output']).toarray()

evaluate_model_graph(X_text_vec, y, X_text_vec2, y_validation_split, X_text_vec3, y_validation_new, models, "Count Vectorizer")